In [1]:
import bs4
import requests
import json
import urllib.request

species = "hsa" #mouse:mmu, human:hsa
r = urllib.request.urlopen('https://www.genome.jp/kegg-bin/show_organism?menu_type=pathway_maps&org=hsa')
items = bs4.BeautifulSoup(r,features="lxml").find_all("b")
group=[x.findNext("ul") for x in items]
total_dict={}
for m in group:
  for i in m.findAll("ul"):
    group_key = i.previous_element.strip().strip()
    total_dict[group_key] = []
    for j in i.findAll("a"):
      pathway_id = j.previous_element.strip()
      pathway_name = j.text
      total_dict[group_key].append((pathway_id,pathway_name))

def get_genes(species,keggid):
    url = "http://togows.dbcls.jp/entry/pathway/{}{}/genes.json".format(species,keggid)
    r = requests.get(url)
    data = json.loads(r.text)
    try:
      gene_names = [i.split(";")[0] for i in data[0].values()]
    except:
      print("error when get genes from {}{}".format(species,keggid))
      return []
    return gene_names

with open("Total_kegg.gmt","w") as fout:
  for k, v in total_dict.items():
    for pid,pname in v:
      pgenes = get_genes(species,pid)
      fout.write("\t".join([pname,pid] + pgenes) + "\n") 